![alt text](https://drive.google.com/uc?export=view&id=1UXScsVx_Wni_JuDdB8LeTnM6jsPfIwkW)

Proprietary content. © Great Learning. All Rights Reserved. Unauthorized use or distribution prohibited.

# Sentiment Classification

### Dataset
- Dataset of 50,000 movie reviews from IMDB, labeled by sentiment positive (1) or negative (0)
- Reviews have been preprocessed, and each review is encoded as a sequence of word indexes (integers).
- For convenience, words are indexed by overall frequency in the dataset, so that for instance the integer "3" encodes the 3rd most frequent word in the data. This allows for quick filtering operations such as: "only consider the top 10,000 most common words, but eliminate the top 20 most common words".
- As a convention, "0" does not stand for a specific word, but instead is used to encode any unknown word.

Command to import data
- `from tensorflow.keras.datasets import imdb`

### Import the data (2 Marks)
- Use `imdb.load_data()` method
- Get train and test set
- Take 10000 most frequent words

In [1]:
#### Add your code here ####
from tensorflow.keras.datasets import imdb
(training_data, training_targets), (testing_data, testing_targets) = imdb.load_data(num_words=10000)

### Pad each sentence to be of same length (2 Marks)
- Take maximum sequence length as 300

In [2]:
#### Add your code here ####
from tensorflow.keras.preprocessing.sequence import pad_sequences
padded_inputs = pad_sequences(training_data, maxlen=300, value = 0.0)
padded_inputs_test = pad_sequences(testing_data, maxlen=300, value = 0.0) 

### Print shape of features & labels (2 Marks)

Number of review, number of words in each review

In [3]:
#### Add your code here ####
padded_inputs.shape

(25000, 300)

In [4]:
#### Add your code here ####
padded_inputs_test.shape

(25000, 300)

Number of labels

In [5]:
#### Add your code here ####
training_targets.shape

(25000,)

In [6]:
#### Add your code here ####
testing_targets.shape

(25000,)

### Print value of any one feature and it's label (2 Marks)

Feature value

In [7]:
#### Add your code here ####
print(padded_inputs[1])

[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    1
  194 1153  194 8255   78  228    5    6 1463 4369 5012  134   26    4
  715    8  118 1634   14  394   20   13  119  954  189  102    5  207
  110 3103   21   14   69  188    8   30   23    7    4  249  126   93
    4  114    9 2300 1523    5  647    4  116    9   35 8163    4  229
    9  340 1322    4  118    9    4  130 4901   19    4 1002    5   89
   29  952   46   37    4  455    9   45   43   38 1543 1905  398    4
 1649 

Label value

In [8]:
#### Add your code here ####
print(training_targets[1])

0


### Decode the feature value to get original sentence (2 Marks)

First, retrieve a dictionary that contains mapping of words to their index in the IMDB dataset

In [9]:
#### Add your code here ####
word_index = imdb.get_word_index()

Now use the dictionary to get the original words from the encodings, for a particular sentence

In [11]:
#### Add your code here ####
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
decoded_review = ' '.join([reverse_word_index.get(i-3, '?') for i in padded_inputs[0]])

decoded_review

"? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert ? is an amazing actor and now the same being director ? father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for ? and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also ? to the two little boy's that played the ? of norman and paul they were just brilliant children are often left out of the ? list i think because the s

Get the sentiment for the above sentence
- positive (1)
- negative (0)

In [12]:
#### Add your code here ####
print(training_targets[0])

1


### Define model (10 Marks)
- Define a Sequential Model
- Add Embedding layer
  - Embedding layer turns positive integers into dense vectors of fixed size
  - `tensorflow.keras` embedding layer doesn't require us to onehot encode our words, instead we have to give each word a unique integer number as an id. For the imdb dataset we've loaded this has already been done, but if this wasn't the case we could use sklearn LabelEncoder.
  - Size of the vocabulary will be 10000
  - Give dimension of the dense embedding as 100
  - Length of input sequences should be 300
- Add LSTM layer
  - Pass value in `return_sequences` as True
- Add a `TimeDistributed` layer with 100 Dense neurons
- Add Flatten layer
- Add Dense layer

In [20]:
#### Add your code here ####
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional, Input,Flatten,Activation
input = Input(shape=(300,))
model = Embedding(input_dim=10000, output_dim=100, input_length=300)(input)
model = Bidirectional(LSTM(units=100, return_sequences=True, recurrent_dropout=0.1))(model)
model = TimeDistributed(Dense(100, activation="softmax"))(model)
model = Flatten()(model)
out = Dense(units=1, activation='sigmoid')(model)
model = Model(input, out)

### Compile the model (2 Marks)
- Use Optimizer as Adam
- Use Binary Crossentropy as loss
- Use Accuracy as metrics

In [21]:
#### Add your code here ####
model.compile(optimizer="Adam", loss="categorical_crossentropy", metrics=["accuracy"])

### Print model summary (2 Marks)

In [22]:
#### Add your code here ####
model.summary() 

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         [(None, 300)]             0         
_________________________________________________________________
embedding_7 (Embedding)      (None, 300, 100)          1000000   
_________________________________________________________________
bidirectional_7 (Bidirection (None, 300, 200)          160800    
_________________________________________________________________
time_distributed_6 (TimeDist (None, 300, 100)          20100     
_________________________________________________________________
flatten (Flatten)            (None, 30000)             0         
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 30001     
Total params: 1,210,901
Trainable params: 1,210,901
Non-trainable params: 0
_________________________________________________

### Fit the model (2 Marks)

In [24]:
#### Add your code here ####
import numpy as np
history = model.fit(padded_inputs, training_targets, batch_size=32, epochs=3, validation_split=0.1, verbose=1)

Epoch 1/3
704/704 [==============================] - 1400s 2s/step - loss: 5.9769e-08 - accuracy: 0.4986 - val_loss: 5.8126e-08 - val_accuracy: 0.5124
Epoch 2/3
704/704 [==============================] - 1286s 2s/step - loss: 5.9769e-08 - accuracy: 0.4986 - val_loss: 5.8126e-08 - val_accuracy: 0.5124
Epoch 3/3
704/704 [==============================] - 1197s 2s/step - loss: 5.9769e-08 - accuracy: 0.4986 - val_loss: 5.8126e-08 - val_accuracy: 0.5124


### Evaluate model (2 Marks)

In [25]:
#### Add your code here ####
print(np.mean(history.history["val_accuracy"]))

0.5123999714851379


### Predict on one sample (2 Marks)

In [26]:
#### Add your code here ####
print(model.predict(padded_inputs_test)[0])

[0.49397016]
